In [2]:
import pandas as pd

# read in all data
test = pd.read_csv('../data/test.txt', delimiter=';', names=['text', 'target'])
train = pd.read_csv('../data/train.txt', delimiter=';',
                    names=['text', 'target'])
val = pd.read_csv('../data/val.txt', delimiter=';', names=['text', 'target'])
trainval = pd.concat([train,val])
testval = pd.concat([test,val])

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# create vectorier for BoW
vectorizer = CountVectorizer(max_df=0.9, min_df=5, stop_words='english', ngram_range=(1, 1))
BoW = vectorizer.fit_transform(trainval.text)
print('Number of Features in BoW: ', len(vectorizer.get_feature_names_out()))

enc = LabelEncoder().fit(['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'])

# transform all the data
X_trainval = vectorizer.transform(trainval.text).toarray()
Y_trainval = enc.transform(trainval.target)
X_train = vectorizer.transform(train.text).toarray()
Y_train = enc.transform(train.target)
X_val = vectorizer.transform(val.text).toarray()
Y_val = enc.transform(val.target)
X_test = vectorizer.transform(test.text).toarray()
Y_test = enc.transform(test.target)

Number of Features in BoW:  3398


In [32]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

clf = XGBClassifier(random_state=0, eval_metric='mlogloss', use_label_encoder=False)

params = {'n_estimators':[10, 50, 100, 250, 500],
          'max_depth':[1, 5, 10, 25, 50],
          'reg_alpha':[0, .1, 1, 10, 100],
          'learning_rate':[.01, .1, .25, .5, 1]
          }

#grid_cv = GridSearchCV(clf,param_grid=params, scoring='f1_macro', verbose=3, n_jobs=1, cv=5)
rand_cv = RandomizedSearchCV(clf, param_distributions=params, n_iter=25, n_jobs=-1, verbose=1, scoring='f1_macro', cv=2, random_state=0)
rand_cv.fit(X_trainval, Y_trainval)


Fitting 2 folds for each of 1 candidates, totalling 2 fits


KeyboardInterrupt: 

In [26]:
print('Best Parameters: ', rand_cv.best_params_)
print('Best Score: ', rand_cv.best_score_)
best_model = rand_cv.best_estimator_

Best Parameters:  {'reg_alpha': 100, 'n_estimators': 50, 'max_depth': 10, 'learning_rate': 1}
Best Score:  0.11208939873309716
